In [130]:
## ADVENT OF CODE 2022, Day 22
## Edmund Dickinson, Python implementation

# File read
input_folder = "Input/"
input_file = "day22a.txt"
file_path = input_folder + input_file

with open(file_path) as file:
    data = file.read()

In [131]:
def parse_map(input):
    data = {}   
    rows = input.splitlines()

    data["ymax"] = len(rows)-1
    data["xmax"] = max([len(row) for row in rows])-1

    for k,row in enumerate(rows):
        y = data["ymax"]-k
        for x in range(data["xmax"]+1):
            if(x > len(row)-1):
                data[(x,y)] = " "
            else:
                data[(x,y)] = row[x]         

    return data

def parse_path(input):
    str = list(input)

    num_buffer = []
    steps = []
    turns = []

    while ( len(str) > 0 ):
        char = str.pop(0)

        if (char == 'L' or char == 'R'):
            steps.append(int("".join(num_buffer)))
            turns.append(char)
            num_buffer = []
        else:
            num_buffer.append(char)
        
    if ( len(num_buffer) > 0 ):
        steps.append(int("".join(num_buffer)))
    
    return steps, turns

def getRow(field, y):
    row = []
    for x in range(field["xmax"]+1):
        row.append(field[(x,y)])
    
    return row

def getCol(field, x):
    col = []
    for y in range(field["ymax"]+1):
        col.append(field[(x,y)])
    
    return col

In [132]:
map_data, path_data = data.split("\n\n")

field = parse_map(map_data)
steps, turns = parse_path(path_data)

In [133]:
def rotate(facing, turn):
    i,j = facing

    if (turn == "L"):
        mult = [-1,1]
    elif (turn == "R"):
        mult = [1,-1]
    else:
        assert False

    return (j*mult[0],i*mult[1])

def advance(field, coord, steps, facing):
    i,j = facing
    x1,y1 = coord

    if (j == 0):
        y2 = y1
        x2 = advance_on_path(getRow(field,y2), x1, steps if i > 0 else -steps)
    elif (i == 0):
        x2 = x1
        y2 = advance_on_path(getCol(field,x2), y1, steps if j > 0 else -steps)
    else:
        assert False

    return (x2, y2)

def advance_on_path(path, init, steps):
    # Steps is negative for reverse direction
    if (steps < 0):
        adv = -1
        steps = -steps
    else:
        adv = 1

    # Count white space left of initial position and discard
    leading_space = path[:init].count(' ')
    trim_path = "".join(path).strip()
    len_trim = len(trim_path)
    pos = init - leading_space

    while (steps > 0):
        new_pos = pos + adv

        # Handle wrapping
        if (new_pos == len_trim):
            new_pos = 0
        elif (new_pos == -1):
            new_pos = len_trim-1

        # Test for blocked
        if (trim_path[new_pos] == '#'):
            steps = 0
        else:
            pos = new_pos
            steps -= 1

    return (pos + leading_space)

def score(field, coord, facing):
    facing_score = {
        (1,0):  0,
        (0,-1): 1,
        (-1,0): 2,
        (0,1):  3
    }

    x,y = coord
    score_row = field["ymax"]-y+1
    score_col = x+1

    print("Row:", score_row, "Col:", score_col)

    return (
        score_row*1000 +
        score_col*4 +
        facing_score[facing]
    )

In [134]:
# Initial position and facing (right)
coord = (getRow(field, field["ymax"]).index('.'),field["ymax"])
facing = (1,0)

for k,step in enumerate(steps):
    # Perform movement
    coord = advance(field, coord, step, facing)

    # Rotate facing
    if (k < len(turns)):
        facing = rotate(facing,turns[k])

print ( score(field, coord, facing) )

Row: 103 Col: 56
103224
